# Finding Baseline Performance

We use the diameter of the nodule as predictor for malignancy as our baseline performance. That is, we assume that larger nodules are more likely to be malignant.

In [82]:
import torch
import matplotlib.pyplot as plt

import src_classification.preprocess_cls as preprocess
import src_classification.model_cls as model
plt.style.use("fivethirtyeight")
# from matplotlib import style
# STYLE = "gt.mplstyle"
# style.use(STYLE)

In [2]:
dataset = preprocess.MalignantLunaDataset(
    val_stride=10,
    is_val_set_bool=True,
)

2024-01-31 10:24:10,580 INFO     src_classification.preprocess_cls:213:__init__ <src_classification.preprocess_cls.MalignantLunaDataset object at 0x14d16447f2d0>: 51583 validation samples, 51429 neg, 154 pos, unbalanced ratio


In [96]:
nodules = dataset.benign_list + dataset.malignant_list
is_malignant = torch.tensor([nodule.is_mal_bool for nodule in nodules])
diameter = torch.tensor([nodule.diameter_mm for nodule in nodules])

number_malignant = is_malignant.sum()
number_benign = len(is_malignant) - number_malignant

In [97]:
threshold = torch.linspace(diameter.max(), diameter.min(),steps=100)

In [98]:
# print(diameter.size())
# print(diameter[None].size())
# print(threshold[:, None].size())
# indexing by None adds a dimension of size 1

# two-dimensional tensor where rows are per threshold and columns are
# per sample information and the value is whethere sample is predicted as positive
predictions = (diameter[None] >= threshold[:, None])
true_positive_diameter = (predictions & is_malignant[None]).sum(1).float() / number_malignant
false_positive_diameter = (predictions & ~is_malignant[None]).sum(1).float() / number_benign

In [103]:
# finding auc using numeric integration by the trapezoidal rule
false_positive_diameter_diff = false_positive_diameter[1:] - false_positive_diameter[:-1]
true_positive_average = (true_positive_diameter[1:] + true_positive_diameter[:-1]) / 2
auc_diameter = (true_positive_average * false_positive_diameter_diff).sum()

# filing for plot
false_positive_fill = torch.ones((false_positive_diameter.shape[0] + 1, ))
false_positive_fill[1:] = false_positive_diameter

true_positive_fill = torch.zeros((true_positive_diameter.shape[0] + 1, ))
true_positive_fill[1:] = true_positive_diameter

In [105]:
plt.figure(figsize=(7,5), dpi=1200)
for i in [62, 88]:
    plt.scatter(false_positive_diameter[i], true_positive_diameter[i], color='red')
    print(f'diam: {round(threshold[i].item(), 2)}, x: {round(false_positive_diameter[i].item(), 2)}, y: {round(true_positive_diameter[i].item(), 2)}')
    
plt.fill(false_positive_fill, true_positive_fill, facecolor='#0077bb', alpha=0.25)
plt.plot(
    false_positive_diameter, 
    true_positive_diameter, 
    label=f'diameter baseline, AUC={auc_diameter:.3f}',
    lw=2,
)
plt.title(f'ROC Diameter Baseline, AUC={auc_diameter:.3f}')
plt.ylabel('True positive rate')
plt.xlabel('False positive rate')
# plt.savefig("images/roc_diameter_baseline.png")

diam: 10.55, x: 0.01, y: 0.56
diam: 5.42, x: 0.43, y: 0.96
